In [1]:
import pandas as pd
import os
import re  # Import regex for pattern replacement

# Yearly merge

In [ ]:
# year='2008'

In [ ]:
for year in range(2008, 2024):
  ##########################################
  # process yield data
  corn_yield = pd.read_csv(f'/content/drive/MyDrive/Lab/BNN/Data/PBCNN/16days/{year}/yield_{year}.csv')
  corn_yield=corn_yield[['Year', 'State ANSI', 'County ANSI', 'County', 'Value']]
  corn_yield=corn_yield.dropna(subset=['County ANSI'])
  corn_yield['GEOID'] = corn_yield['State ANSI'].astype(int).astype(str).str.zfill(2) + corn_yield['County ANSI'].astype(int).astype(str).str.zfill(3)
  corn_yield=corn_yield.sort_values(by=['GEOID'])
  corn_yield['GEOID'] = corn_yield['GEOID'].astype(str)
  # corn_yield

  ##########################################################################################
  # process other data
  folder_path = f"/content/drive/MyDrive/Lab/BNN/Data/PBCNN/16days/{year}/other"
  csv_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".csv")])

  corn_gee_other = None

  for file in csv_files:
      file_path = os.path.join(folder_path, file)
      corn_gee = pd.read_csv(file_path)

      corn_gee.drop(columns=['system:index', '.geo'], inplace=True)
      # Extract "GEOID" and "NAME" columns
      first_columns = ["GEOID", "NAME"]
      # Extract numeric columns and sort them in ascending order
      numeric_columns = sorted([col for col in corn_gee.columns if col.isdigit()], key=int)
      # Combine them in the desired order
      sorted_columns = first_columns + numeric_columns
      # Reorder the DataFrame
      corn_gee = corn_gee[sorted_columns]
      corn_gee=corn_gee.sort_values(by=['GEOID'])

      corn_gee['GEOID'] = corn_gee['GEOID'].astype(str)

      # Extract the suffix from the file name
      suffix = os.path.splitext(file)[0]

      # Rename columns by adding the suffix
      corn_gee = corn_gee.rename(columns=lambda x: f"{x}_{suffix}" if x not in ['GEOID', 'NAME'] else x)

      if corn_gee_other is None:
          corn_gee_other = corn_gee
      else:
          corn_gee_other = pd.merge(corn_gee_other, corn_gee, on=['GEOID', 'NAME'], how='right')

  # corn_gee_other.head()

  ###########################################################################################################
  # # process soil data
  # folder_path = f"/content/drive/MyDrive/Lab/BNN/Data/PBCNN/16days/{year}/soil"
  # csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

  # corn_gee_soil = None

  # for file in csv_files:
  #     file_path = os.path.join(folder_path, file)
  #     corn_gee = pd.read_csv(file_path)
  #     corn_gee.drop(columns=['system:index', '.geo'], inplace=True)

  #     corn_gee=corn_gee.sort_values(by=['GEOID'])

  #     corn_gee['GEOID'] = corn_gee['GEOID'].astype(str)

  #     # Extract the suffix from the file name
  #     suffix = os.path.splitext(file)[0]

  #     # Rename columns by adding the suffix
  #     corn_gee = corn_gee.rename(columns=lambda x: f"{x}_{suffix}" if x not in ['GEOID', 'NAME'] else x)

  #     if corn_gee_soil is None:
  #         corn_gee_soil = corn_gee
  #     else:
  #         corn_gee_soil = pd.merge(corn_gee_soil, corn_gee, on=['GEOID', 'NAME'], how='right')

  # corn_gee_soil['GEOID'] = corn_gee_soil['GEOID'].astype(str)
  # corn_gee_soil.head()

  #####################################################################################################
  # final merge
  # corn_gee_all = pd.merge(corn_gee_other, corn_gee_soil, on=['GEOID', 'NAME'], how='right')
  # corn_gee_final = pd.merge(corn_gee_all, corn_yield[['GEOID', 'Value']], on='GEOID', how='right')
  # corn_gee_final = corn_gee_final.dropna()
  # corn_gee_final.insert(0, 'year', year)
  # corn_gee_final

  #no soil
  corn_gee_final = pd.merge(corn_gee_other, corn_yield[['GEOID', 'Value']], on='GEOID', how='right')
  corn_gee_final = corn_gee_final.dropna()
  corn_gee_final.insert(0, 'year', year)
  # corn_gee_final

  # export
  corn_gee_final.reset_index(drop=True, inplace=True)
  corn_gee_final.to_csv(f'/content/drive/MyDrive/Lab/BNN/Data/PBCNN/16days/{year}/gee/corn_gee_all_{year}.csv', index=False)

In [ ]:
# # export
# corn_gee_final.reset_index(drop=True, inplace=True)
# corn_gee_final.to_csv(f'/content/drive/MyDrive/Lab/BNN/Data/PBCNN/16days/{year}/gee/corn_gee_all_{year}.csv', index=False)

# Merge

In [10]:
# Define the range of years
years = range(2008, 2024)

# Initialize an empty list to store DataFrames
df_list = []

for year in years:
    folder_path = f"/content/drive/MyDrive/Lab/BNN/Data/PBCNN/16days/{year}/gee/"

    # Check if the folder exists
    if os.path.exists(folder_path):
        csv_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".csv")])

        # Read and append CSV files one by one
        for file in csv_files:
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)

            # Rename columns: remove year suffix (4-digit number) at the end
            df.columns = [re.sub(r'_\d{4}$', '', col) for col in df.columns]

            df_list.append(df)  # Store DataFrame in the list

# Concatenate all DataFrames at once for efficiency
if df_list:  # Ensure there is data before concatenating
    df_final = pd.concat(df_list, ignore_index=True)
    print(df_final.info())  # Print summary
else:
    print("No CSV files found.")

# df_final['GEOID'] = df['GEOID'].astype('int64')
# Display summary of the final DataFrame
df_final
print(df_final['GEOID'].dtypes)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13205 entries, 0 to 13204
Columns: 378 entries, year to Value
dtypes: float64(375), int64(2), object(1)
memory usage: 38.1+ MB
None
int64


# Historical yield

In [13]:
# Define the range of end years you want to loop through
end_year_list = range(2008, 2024)

# Initialize list to store intermediate DataFrames
all_records = []

for end_year in end_year_list:
    start_year = end_year - 5
    print(f"Averaging years: {list(range(start_year, end_year))}")  # <-- Added print here

    merged_df = None

    for year in range(start_year, end_year):
        # Read and process yield data for the year
        corn_yield = pd.read_csv(f'/content/drive/MyDrive/Lab/BNN/Data/PBCNN/yield/yield_{year}.csv')
        corn_yield = corn_yield[['Year', 'State ANSI', 'County ANSI', 'County', 'Value']]
        corn_yield = corn_yield.dropna(subset=['County ANSI'])

        corn_yield['GEOID'] = corn_yield['State ANSI'].astype(int).astype(str).str.zfill(2) + \
                              corn_yield['County ANSI'].astype(int).astype(str).str.zfill(3)
        corn_yield = corn_yield.sort_values(by=['GEOID'])
        corn_yield['GEOID'] = corn_yield['GEOID'].astype(str)

        # Rename the 'Value' column to include the year as a suffix
        corn_yield = corn_yield[['GEOID', 'Value']].rename(columns={'Value': f'Yield_{year}'})

        # Merge into the main DataFrame
        if merged_df is None:
            merged_df = corn_yield
        else:
            merged_df = pd.merge(merged_df, corn_yield, on='GEOID', how='outer')

    # Drop rows with any missing values
    merged_df = merged_df.dropna()

    # Calculate average yield across the 5-year period
    yield_columns = [f'Yield_{year}' for year in range(start_year, end_year)]
    merged_df['Hist_Yield'] = merged_df[yield_columns].mean(axis=1)

    # Add the year column
    merged_df['year'] = end_year  # represents last year in 5-year window

    # Keep only relevant columns
    result_df = merged_df[['GEOID', 'year', 'Hist_Yield']]

    # Store the result
    all_records.append(result_df)

# Combine all 5-year average windows
final_df_yield = pd.concat(all_records, ignore_index=True)

# Sort by GEOID and Year
final_df_yield = final_df_yield.sort_values(by=['GEOID', 'year'])
final_df_yield = final_df_yield.dropna()

# Display the result
final_df_yield['GEOID'] = final_df_yield['GEOID'].astype('int64')
final_df_yield


Averaging years: [2003, 2004, 2005, 2006, 2007]
Averaging years: [2004, 2005, 2006, 2007, 2008]
Averaging years: [2005, 2006, 2007, 2008, 2009]
Averaging years: [2006, 2007, 2008, 2009, 2010]
Averaging years: [2007, 2008, 2009, 2010, 2011]
Averaging years: [2008, 2009, 2010, 2011, 2012]


<ipython-input-13-98a18c72dcf9>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corn_yield['GEOID'] = corn_yield['State ANSI'].astype(int).astype(str).str.zfill(2) + \


Averaging years: [2009, 2010, 2011, 2012, 2013]
Averaging years: [2010, 2011, 2012, 2013, 2014]
Averaging years: [2011, 2012, 2013, 2014, 2015]
Averaging years: [2012, 2013, 2014, 2015, 2016]
Averaging years: [2013, 2014, 2015, 2016, 2017]
Averaging years: [2014, 2015, 2016, 2017, 2018]
Averaging years: [2015, 2016, 2017, 2018, 2019]
Averaging years: [2016, 2017, 2018, 2019, 2020]
Averaging years: [2017, 2018, 2019, 2020, 2021]
Averaging years: [2018, 2019, 2020, 2021, 2022]


,GEOID,year,Hist_Yield
0,1001,2008,82.40
1,1003,2008,111.60
8147,1003,2014,126.22
9306,1003,2015,129.20
10450,1003,2016,126.26
...,...,...,...
11502,56021,2016,117.06
12522,56021,2017,121.62
1737,56029,2008,137.80
3205,56029,2009,139.20


In [14]:
print(final_df_yield['GEOID'].dtypes)

int64


# Final merge

In [15]:
final_df_ultimate = pd.merge(df_final, final_df_yield, on=['GEOID', 'year'], how='right')
final_df_ultimate=final_df_ultimate.dropna()
final_df_ultimate

,year,GEOID,NAME,0_EVI2_max,16_EVI2_max,32_EVI2_max,48_EVI2_max,64_EVI2_max,80_EVI2_max,96_EVI2_max,...,224_vpdmin_mean,240_vpdmin_mean,256_vpdmin_mean,272_vpdmin_mean,288_vpdmin_mean,304_vpdmin_mean,320_vpdmin_mean,336_vpdmin_mean,Value,Hist_Yield
1189,2008,17001,Adams,0.147654,0.151047,0.148280,0.142598,0.145347,0.170781,0.210848,...,1.200540,0.728699,0.902163,0.987076,1.050679,1.100386,0.699644,0.534708,171.0,157.20
1190,2009,17001,Adams,0.147307,0.145784,0.139097,0.135640,0.144963,0.170189,0.190185,...,0.835620,0.723565,0.716691,0.431606,0.554339,1.189545,0.586923,0.458003,172.0,157.20
1191,2010,17001,Adams,0.008479,0.022451,0.032183,0.030056,0.149444,0.188879,0.221794,...,1.412100,1.387670,0.584268,1.263111,1.590681,1.482043,0.609211,0.371121,121.9,152.60
1192,2011,17001,Adams,0.143482,0.001059,0.129279,0.143183,0.164070,0.210361,0.250294,...,1.937940,2.452674,1.159089,1.642889,1.196471,1.125690,0.901277,0.471140,135.8,151.18
1193,2012,17001,Adams,0.128778,0.122583,0.129690,0.140300,0.180121,0.230268,0.249973,...,2.458204,1.916287,1.107186,1.226567,1.126570,1.106573,0.793086,0.672355,76.0,152.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17713,2019,55141,Wood,0.153219,0.074173,0.003484,0.001202,0.004188,0.148894,0.180138,...,0.639609,0.479540,0.469657,0.552220,0.437841,0.418246,0.259752,0.274270,141.0,148.78
17714,2020,55141,Wood,0.004204,0.013542,0.004005,-0.003569,0.064604,0.185710,0.215225,...,0.840889,0.687758,0.726022,0.737123,0.456606,0.679853,0.493419,0.482009,160.9,147.26
17715,2021,55141,Wood,0.014139,0.024980,0.010085,0.000908,0.166163,0.176720,0.248049,...,0.863211,0.692707,0.676994,0.397366,0.393366,0.557531,0.574497,0.321148,172.1,150.64
17716,2022,55141,Wood,0.018572,0.012713,0.004286,0.013932,0.160730,0.169575,0.177021,...,0.355741,0.499815,0.373726,0.435670,0.635327,0.865689,0.316405,0.344853,167.9,153.64


In [16]:
# Reorder columns to have 'year', 'GEOID', 'NAME', and 'Hist_Yield' first
columns = ['year', 'GEOID', 'NAME', 'Hist_Yield'] + [col for col in final_df_ultimate.columns if col not in ['year', 'GEOID', 'NAME', 'Hist_Yield']]

# Reapply the column order
final_df_ultimate = final_df_ultimate[columns]
# Sort by 'GEOID' and 'year'
final_df_ultimate = final_df_ultimate.sort_values(by=['year', 'GEOID'])
# Display the final DataFrame
final_df_ultimate

,year,GEOID,NAME,Hist_Yield,0_EVI2_max,16_EVI2_max,32_EVI2_max,48_EVI2_max,64_EVI2_max,80_EVI2_max,...,208_vpdmin_mean,224_vpdmin_mean,240_vpdmin_mean,256_vpdmin_mean,272_vpdmin_mean,288_vpdmin_mean,304_vpdmin_mean,320_vpdmin_mean,336_vpdmin_mean,Value
1189,2008,17001,Adams,157.20,0.147654,0.151047,0.148280,0.142598,0.145347,0.170781,...,1.441083,1.200540,0.728699,0.902163,0.987076,1.050679,1.100386,0.699644,0.534708,171.0
1203,2008,17003,Alexander,158.00,0.189856,0.182003,0.192778,0.193562,0.207553,0.211543,...,1.071237,1.061594,0.949244,0.918486,0.544681,1.039416,0.730738,0.785054,0.621747,144.0
1209,2008,17005,Bond,129.80,0.198827,0.184416,0.180175,0.177961,0.193882,0.235026,...,0.920327,0.902167,0.921507,0.555067,0.744526,0.841070,0.799324,0.650785,0.506167,164.0
1215,2008,17007,Boone,159.40,0.048590,0.064420,-0.021191,-0.007759,0.117175,0.135432,...,0.982886,1.102153,1.006634,0.624747,0.859295,0.568203,0.764125,0.478728,0.299500,169.0
1226,2008,17009,Brown,155.60,0.150895,0.145026,0.151436,0.144278,0.143027,0.165326,...,0.661972,0.547623,0.393235,0.449751,0.618618,0.825201,0.863943,0.537787,0.509684,152.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17658,2023,55133,Waukesha,176.60,0.170489,0.166698,0.157492,0.156371,0.172940,0.190284,...,0.408122,0.683095,0.804246,0.645962,0.602257,0.394258,0.816662,0.434729,0.446472,172.4
17674,2023,55135,Waupaca,160.40,0.041582,0.043505,0.097437,0.096768,0.017334,0.168077,...,0.723374,0.785403,0.863435,0.373686,0.559184,0.356186,0.773756,0.541861,0.459983,162.4
17690,2023,55137,Waushara,169.94,0.034545,0.032347,0.118773,0.118579,0.031049,0.176987,...,0.721384,0.707198,0.802921,0.308908,0.543998,0.323624,0.711772,0.449010,0.403658,179.7
17701,2023,55139,Winnebago,159.16,0.021353,0.023754,0.075050,0.093029,0.046166,0.151742,...,1.167329,1.191488,1.181820,0.738205,1.051220,0.445141,1.021652,0.599678,0.537731,176.1


In [17]:
# export
final_df_ultimate.reset_index(drop=True, inplace=True)
final_df_ultimate.to_csv(f'/content/drive/MyDrive/Lab/BNN/Data/PBCNN/16days/corn_input_final.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Lab/BNN/Data/PBCNN/16days/corn_input_final.csv')

import pandas as pd

# Convert to normal decimal notation
df = df.map(lambda x: '{:.5f}'.format(x) if isinstance(x, float) else x)
X = df.iloc[1:, 3:-1].values.astype(np.float32)
# Count NaN values
print(X.shape)
print("Number of NaNs:", np.isnan(X).sum())
# # Print indices of NaN values
# nan_indices = np.where(np.isnan(X))[0]
# print("Indices of NaNs:", nan_indices)
# print("nan values", X[nan_indices])

(13204, 374)
Number of NaNs: 0


In [ ]:
# export
df.reset_index(drop=True, inplace=True)
df.to_csv(f'/content/drive/MyDrive/Lab/BNN/Data/PBCNN/16days/corn_input_decimal_round.csv', index=False)